In [466]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
pd.options.display.max_rows = 400
pd.options.display.max_columns = 100
import warnings
warnings.filterwarnings('ignore')

### ETL eCAP

In [244]:
pats = pd.read_csv('patients.csv')
# ups = pd.read_csv('ups.csv')
# uabs = pd.read_csv('uabs.csv')
# uis = pd.read_csv('uis.csv')
# pccmaca = pd.read_csv('pccmaca.csv')

In [245]:
# pats2 = pd.merge(pats, ups[['UP','EAP']], on='UP', how='left')
# pats3 = pd.merge(pats2, uabs[['UAB_SK','METGE']], on='UAB_SK', how='left')
# pats4 = pd.merge(pats3, uis[['UI_SK','INFERMERA']], on='UI_SK', how='left')
# pats5 = pd.merge(pats4, pccmaca[['CIP','DESC_PACIENT']], on='CIP', how='left')
# pats5.shape[0]/pats.shape[0]

### ETL BO

In [478]:
def get_tam_calendar():

    d1 = datetime.now().date()
    d2 = d1.replace(day=1)
    d_last = d2 - timedelta(days=1)
    d3 = d_last - pd.DateOffset(years=1)
    d_first = d3 + timedelta(days=1)

    calendar_array = pd.date_range(start=d_first, end=d_last, freq='D')
    calendar_df = pd.DataFrame(calendar_array, columns=['DATA'])

    return calendar_df
    
def add_cips(df, df2):

    df_cips = df[df['CIP'].notnull()]
    df_nhcs = pd.merge(df[df['CIP'].isnull()].drop('CIP', axis=1),
                       df2[['CIP','NHC']],
                       on='NHC',
                       how='left')
    df3 = pd.concat([df_cips, df_nhcs]).reset_index(drop=True)
    df3['CIP_CDT'] = np.where(df3['CIP'].isin(tuple(set(df2['CIP']))), 1, 0)

    return df3
    
def get_urg(fname, df_ecap):
    
    df = pd.read_csv(fname).rename({'CIP (13d)':'CIP',
                                    'Pacient (NHC)':'NHC',
                                    'Episodi':'URG_EPISODI',
                                    'Edat':'URG_EDAT',
                                    'Procedencia desc':'URG_PROCEDENCIA',
                                    'Centre origen desc':'URG_CENTRE_PROCEDENCIA',
                                    'Forma arribar pacient desc':'URG_MEDI_PROCEDENCIA',
                                    'Data hora entrada':'URG_DH_ENTRADA',
                                    'Data hora assistencia':'URG_DH_ASSISTENCIA',
                                    'Motiu urgència codi':'URG_MOTIU',
                                    'Triatge desc (Darrer)':'URG_TRIATGE',
                                    'Servei alta desc':'URG_SERVEI',
                                    'Diagnòstic codi':'URG_DIAG_CODI',
                                    'Diagnòstic descripció':'URG_DIAG',
                                    'És diagnòstic principal de servei mèdic':'URG_DIAG_PPAL',
                                    'Data hora sortida':'URG_DH_SORTIDA',
                                    'Classe fi episodi desc':'URG_DESTI',
                                    'Centre destí desc':'URG_CENTRE_DESTI'}, axis=1)

    df2 = df.sort_values(['URG_EPISODI',
                          'URG_DIAG_PPAL']).drop_duplicates('URG_EPISODI', keep='last').reset_index(drop=True) 

    df2['URG_MOTIU'] = np.where(df2['URG_MOTIU']=='01','Malaltia comú',
                        np.where(df2['URG_MOTIU']=='02','Accident',
                        np.where(df2['URG_MOTIU']=='03','Agressió/Autolisi',
                        np.where(df2['URG_MOTIU']=='04','Part','Altres'))))

    df2['URG_D_ENTRADA'] = pd.to_datetime(df2['URG_DH_ENTRADA']).dt.date
    df2['URG_H_ENTRADA'] = pd.to_datetime(df2['URG_DH_ENTRADA']).dt.hour
    df2['URG_D_SORTIDA'] = pd.to_datetime(df2['URG_DH_SORTIDA']).dt.date
    df2['URG_H_SORTIDA'] = pd.to_datetime(df2['URG_DH_SORTIDA']).dt.hour

    df3 = add_cips(df2, df_ecap)

    del df, df2, df_ecap

    return df3

def get_ssf(fname, df_ecap):
    
    df = pd.read_csv(fname).rename({'CIP (13d)':'CIP',
                                    'Pacient (NHC)':'NHC',
                                    'Episodi':'SSF_EPISODI',
                                    'Edat (number)':'SSF_EDAT',
                                    'Tipus episodi desc':'SSF_EPISODI_TIPUS',
                                    'Classe admissió desc':'SSF_ADMISSIO_TIPUS',
                                    'Origen admisió desc':'SSF_PROCEDENCIA',
                                    'Centre origen desc':'SSF_CENTRE_PROCEDENCIA',
                                    'Servei admissió desc':'SSF_ADMISSIO_SERVEI',
                                    'UT admissió dec':'SSF_ADMISSIO_UT',
                                    'Data ingres':'SSF_DH_INGRES',
                                    'Data hora alta':'SSF_DH_ALTA',
                                    "Estada de l'episodi (dies)":'SSF_D_ESTADA',
                                    'GRD pes':'SSF_GRD_PES',
                                    'Diagnòstic principal cataleg':'SSF_DIAG_CAT',
                                    'Diagnòstic principal codi':'SSF_DIAG_PPAL_CODI',
                                    'Diagnòstic principal desc':'SSF_DIAG_PPAL',
                                    'Servei alta desc':'SSF_ALTA_SERVEI',
                                    'UT alta desc':'SSF_ALTA_UT',
                                    'Metge alta desc':'SSF_ALTA_METGE',
                                    'Classe alta desc':'SSF_ALTA_DESTI',
                                    'Centre destí alta desc':'SSF_ALTA_CENTRE_DESTI'},
                                    axis=1).drop_duplicates(keep='last').reset_index(drop=True)

    df = df[df['SSF_EDAT'].notnull()].reset_index(drop=True)

    df['SSF_EDAT'] = np.where(df['SSF_EDAT']=='0', 0,
                              df['SSF_EDAT'].str.split(',', expand=True)[0].astype(int))

    df['SSF_D_INGRES'] = pd.to_datetime(df['SSF_DH_INGRES']).dt.date
    df['SSF_H_INGRES'] = pd.to_datetime(df['SSF_DH_INGRES']).dt.hour
    df['SSF_D_ALTA'] = pd.to_datetime(df['SSF_DH_ALTA']).dt.date
    df['SSF_H_ALTA'] = pd.to_datetime(df['SSF_DH_ALTA']).dt.hour

    df3 = add_cips(df, df_ecap)

    del df, df_ecap

    return df3

def get_jx(fname, df_ecap):
    
    df = pd.read_csv(fname).rename({'CIP (13d)':'CIP',
                                    'Pacient (NHC)':'NHC',
                                    'Episodi':'JX_EPISODI',
                                    'Edat (number)':'JX_EDAT',
                                    'Tipus episodi desc':'JX_EPISODI_TIPUS',
                                    'Classe admissió desc':'JX_ADMISSIO_TIPUS',
                                    'Origen admisió desc':'JX_PROCEDENCIA',
                                    'Centre origen desc':'JX_CENTRE_PROCEDENCIA',
                                    'Servei admissió desc':'JX_ADMISSIO_SERVEI',
                                    'UT admissió dec':'JX_ADMISSIO_UT',
                                    'Data ingres':'JX_DH_INGRES',
                                    'Data hora alta':'JX_DH_ALTA',
                                    "Estada de l'episodi (dies)":'JX_D_ESTADA',
                                    'GRD pes':'JX_GRD_PES',
                                    'Diagnòstic principal cataleg':'JX_DIAG_CAT',
                                    'Diagnòstic principal codi':'JX_DIAG_PPAL_CODI',
                                    'Diagnòstic principal desc':'JX_DIAG_PPAL',
                                    'Servei alta desc':'JX_ALTA_SERVEI',
                                    'UT alta desc':'JX_ALTA_UT',
                                    'Metge alta desc':'JX_ALTA_METGE',
                                    'Classe alta desc':'JX_ALTA_DESTI',
                                    'Centre destí alta desc':'JX_ALTA_CENTRE_DESTI'},
                                    axis=1).drop_duplicates(keep='last').reset_index(drop=True)

    df = df[df['JX_EDAT'].notnull()].reset_index(drop=True)

    df['JX_EDAT'] = np.where(df['JX_EDAT']=='0', 0,
                             df['JX_EDAT'].str.split(',', expand=True)[0].astype(int))

    df['JX_D_INGRES'] = pd.to_datetime(df['JX_DH_INGRES']).dt.date
    df['JX_H_INGRES'] = pd.to_datetime(df['JX_DH_INGRES']).dt.hour
    df['JX_D_ALTA'] = pd.to_datetime(df['JX_DH_ALTA']).dt.date
    df['JX_H_ALTA'] = pd.to_datetime(df['JX_DH_ALTA']).dt.hour

    df3 = add_cips(df, df_ecap)

    del df, df_ecap

    return df3

def get_pxs(fname, df2):

    df = pd.read_csv(fname).rename({'Episodi':'JX_EPISODI',
                                    'Data Inici Moviment':'PXS_DH_IM',
                                    'Data Alta':'PXS_DH_FM',
                                    'Servei Hospitalitzacio Descripció':'PXS_SERVEI',
                                    'UP del servei gestor desc':'PXS_SERVEI_UP',
                                    'Unitat Tractament Descripcio':'PXS_UT',
                                    'UP de la UT desc':'PXS_UT_UP'},
                                    axis=1).drop_duplicates(keep='last').reset_index(drop=True)

    df3 = df[(df['JX_EPISODI'].isin(tuple(set(df2['JX_EPISODI'])))) & (
        ~df['PXS_DH_FM'].str.contains('9999'))].sort_values(['JX_EPISODI','PXS_DH_FM']).reset_index(drop=True)
    df3['PXS_MOV_NUM'] = df3.groupby('JX_EPISODI').cumcount() + 1
    df3['PXS_PK'] = df3['JX_EPISODI'].astype(str) + '_' + df3['PXS_MOV_NUM'].astype(str).str.pad(
        width=2, side='left', fillchar='0')

    df3['PXS_D_IM'] = pd.to_datetime(df3['PXS_DH_IM']).dt.date
    df3['PXS_H_IM'] = pd.to_datetime(df3['PXS_DH_IM']).dt.hour
    df3['PXS_D_FM'] = pd.to_datetime(df3['PXS_DH_FM']).dt.date
    df3['PXS_H_FM'] = pd.to_datetime(df3['PXS_DH_FM']).dt.hour

    del df, df2

    return df3


In [479]:
%%time

urg = get_urg('bo_urg.csv', pats)
ssf = get_ssf('bo_ssf.csv', pats)
jx = get_jx('bo_jx.csv', pats)
pxs = get_pxs('bo_pxs.csv', jx)
calendar = get_tam_calendar()

CPU times: user 23 s, sys: 177 ms, total: 23.2 s
Wall time: 23.9 s


In [458]:
print(urg['URG_EPISODI'].nunique()/urg.shape[0])
print(ssf['SSF_EPISODI'].nunique()/ssf.shape[0])
print(jx['JX_EPISODI'].nunique()/jx.shape[0])
print(pxs['PXS_PK'].nunique()/pxs.shape[0])

1.0
1.0
1.0
1.0


In [480]:
%%time

# urg.to_csv('urg.csv', index=False)
# ssf.to_csv('ssf.csv', index=False)
# jx.to_csv('jx.csv', index=False)
# pxs.to_csv('pxs.csv', index=False)
calendar.to_csv('tam.csv', index=False)

CPU times: user 2.5 ms, sys: 10 ms, total: 12.5 ms
Wall time: 13.8 ms


In [471]:
calendar['DATA'].min()

Timestamp('2023-10-01 00:00:00')

In [477]:
calendar.dtypes

DATA       datetime64[ns]
PERIODE            object
dtype: object

In [481]:
calendar.head(1)

,DATA
0,2023-10-01


In [469]:
jx['CIP_CDT'].value_counts()

CIP_CDT
1    18967
0     3222
Name: count, dtype: int64

### Mirant episodis falsos ssf

In [358]:
# ssf2 = ssf.sort_values(['NHC','SSF_DH_INGRES']).reset_index(drop=True)
# ssf2['HDIFF'] = np.where(ssf2['NHC']==ssf2['NHC'].shift(1),
#                          (pd.to_datetime(ssf2['SSF_DH_INGRES'])-pd.to_datetime(
#                              ssf2['SSF_DH_ALTA'].shift(1))).dt.total_seconds()/3600,
#                          np.nan)

In [359]:
# ssf2[ssf2['HDIFF']<1]

In [447]:
pxs['PXS_MOV_NUM_2'] = pxs.groupby(['JX_EPISODI','PXS_UT']).cumcount() + 1

In [451]:
pxs[pxs['JX_EPISODI']==18695831]['PXS_SERVEI'].value_counts()

PXS_SERVEI
NEUROCIRURGIA                    9
ANESTESIOLOGIA I REANIMACIO      3
MEDICINA FISICA REHABILITACIO    3
Name: count, dtype: int64